In [4]:
# 허깅페이스의 transformer를 사용하기 위함 
# !pip install transformers

In [1]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

Tokenizer는 모델에 어떠한 입력을 넣어주기 위해서 전처리를 담당합니다. <br>
토크나이저는 허깅페이스의 PreTrainedTokenizer 인 GPT2Tokenizer 를 사용합니다. Tokenizer들은 크게 3가지 기능을 제공합니다.<br>

- Tokenizing : 입력 문자열을 token id로 변환(encoding), token id를 다시 문자열로 변환(decoding)의 기능<br>
- 기존의 구조(BPE, Sentencepiece 등)에 독립적으로 추가적인 token들을 추가하는 기능<br>
- Special token들을 (mask, BOS, EOS 등) 관리하는 기능<br>

In [ ]:
BOS = "</s>" # 문장의 시작을 나타내는 토큰
EOS = "</s>" # 문장의 끝을 나타내는 토큰
PAD = "<pad>" # 동일한 Batch 내에서 입력의 크기를 동일하게 하기 위해서 사용하는 토큰
MASK = "<unused0>" 

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

---
KoGPT2 모델 불러와서 사용해보기

In [3]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [8]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


In [5]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

text = 'Q. 대한민국 예산군의 전통 시장에 대해서 알려줘'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

Q. 대한민국 예산군의 전통 시장에 대해서 알려줘야겠다.
이번에는 우리 군의 재래시장 현대화 사업에 대해 알아보도록 하겠다.
우리 군은 지난해부터 시장 상인회와 함께 상인들의 의견을 수렴하여 사업을 추진해왔다.
그 결과 올해는 총 사업비 4억 원을 투입, 기존 시장을 리모델링하고 아케이드와 주차장을 조성하는 등 현대식 시장으로 탈바꿈했다.
또한 지역 내 우수 농특산물을 활용한 다양한 먹거리 장터를 운영함으로써 소비자들에게 보다 많은 볼거리를 제공할 수 있게 되었다.
특히 이번 사업은 군에서 추진하는 사업으로 전국 최초로 시행하는 것으로 그 의미가 크다.
앞으로 이 사업이 성공적으로 마무리되면 침체된 경기 활성화에 큰 도움이 될


In [19]:
여행 일정 : 2 박 3일
어느지역 : 예산군
누구랑 : 가족들이랑
    


'd:\\save_desktop\\study_project'

In [ ]:
# s 값을 가장 작게 하기 위한 A의 수를 재배열
# A의 수랑 B의 수를 곱하고 그것들을 더해주는 
# 이 수를 가장 작게 하기 위해서는 정렬을 한다

import sys
input = sys.stdin.readline

cnt_num = int(input())
A = list(map(int,input().split())).sort()
B = list(map(int,input().split())).sort()


answer = 0
for a,b in zip(A,B):
    data = a*b
    answer += data

print(answer)

In [33]:
import sys,io
file = open("inputs.txt",'w')
data='''3 4
ohhenrie
charlie
baesangwook
obama
baesangwook
ohhenrie
clinton
'''

file.write(data)
file.close()
input_file = open('inputs.txt',"r")
sys.stdin = io.StringIO(input_file.read())

In [34]:
input = sys.stdin.readline

N,M = map(int,input().split())

nosee = dict()
for _ in range(N):
    name = input().rstrip()
    nosee[f'{name}'] = 0

answer = []
for _ in range(M):
    name = input().rstrip()
    if name in nosee:
        answer.append(name)

print(len(answer))

for name_ in answer:
    print(name_)
# # N = 듣도 M = 보도B
# if N > M:

nosee

2
baesangwook
ohhenrie


{'ohhenrie': 0, 'charlie': 0, 'baesangwook': 0}

In [3]:
print(nolisten)

['obama\n', 'baesangwook\n', 'ohhenrie\n', 'clinton\n']
